# Import Libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Clone Dataset

In [ ]:
! git clone https://github.com/education454/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 2301, done.
remote: Total 2301 (delta 0), reused 0 (delta 0), pack-reused 2301
Receiving objects: 100% (2301/2301), 1.31 GiB | 36.20 MiB/s, done.
Checking out files: 100% (2295/2295), done.


In [ ]:
base_dir = '/content/datasets/Data'

train_dir = os.path.join(base_dir, 'train')
train_covid_dir = os.path.join(train_dir, 'COVID19')
train_normal_dir = os.path.join(train_dir, 'NORMAL')

test_dir = os.path.join(base_dir, 'test')
test_covid_dir = os.path.join(test_dir, 'COVID19')
test_normal_dir = os.path.join(test_dir, 'NORMAL')

In [ ]:
train_covid_names = os.listdir(train_covid_dir)
print(train_covid_names[:10])

train_normal_names = os.listdir(train_normal_dir)
print(train_normal_names[:10])

test_covid_names = os.listdir(test_covid_dir)
print(test_covid_names[:10])

test_normal_names = os.listdir(test_normal_dir)
print(test_normal_names[:10])

['COVID19(381).jpg', 'COVID19(273).jpg', 'COVID19(137).jpg', 'COVID19(213).jpg', 'COVID19(480).jpg', 'COVID19(87).jpg', 'COVID19(401).jpg', 'COVID19(543).jpg', 'COVID19(475).jpg', 'COVID19(120).jpg']
['NORMAL(1401).jpg', 'NORMAL(35).jpg', 'NORMAL(148).jpg', 'NORMAL(198).jpg', 'NORMAL(268).jpg', 'NORMAL(19).jpg', 'NORMAL(25).jpg', 'NORMAL(1192).jpg', 'NORMAL(426).jpg', 'NORMAL(506).jpg']
['COVID-19 (718).jpg', 'COVID19(200).jpg', 'COVID19(463).jpg', 'COVID19(315).jpg', 'COVID19(561).jpg', 'COVID19(450).jpg', 'COVID19(186).jpg', 'COVID19(426).jpg', 'COVID-19 (587).jpg', 'COVID-19 (789).jpg']
['NORMAL(632).jpg', 'NORMAL(327).jpg', 'NORMAL(687).jpg', 'NORMAL(1301).jpg', 'NORMAL(329).jpg', 'NORMAL(484).jpg', 'NORMAL(455).jpg', 'NORMAL(18).jpg', 'NORMAL(155).jpg', 'NORMAL(187).jpg']


In [ ]:
print('Train Dataset Covid Images:', len(train_covid_names))
print('Train Dataset Normal Images:', len(train_normal_names))

print('Test Dataset Covid Images:', len(test_covid_names))
print('Test Dataset Normal Images:', len(test_normal_names))

print('Total Train Images:', len(train_covid_names + train_normal_names))
print('Total Test Images:', len(test_covid_names + test_normal_names))

Train Dataset Covid Images: 545
Train Dataset Normal Images: 1266
Test Dataset Covid Images: 167
Test Dataset Normal Images: 317
Total Train Images: 1811
Total Test Images: 484


# Data Visualization

In [ ]:
# set the number of columns and rows
nrows = 4
ncols = 4

# set the figure size
fig = plt.gcf()
fig.set_size_inches(12, 12)

# get the filenames from the covid & normal dir of the train dataset
next_covid_pic = [os.path.join(train_covid_dir, fname) for fname in train_covid_names[0:8]]
next_normal_pic = [os.path.join(train_normal_dir, fname) for fname in train_normal_names[0:8]]

# print the list
print(next_covid_pic)
print(next_normal_pic)

for i, img_path in enumerate(next_covid_pic + next_normal_pic):
  data = img_path.split('/', 6)[6]
  sp = plt.subplot(nrows, ncols, i+1)
  sp.axis('off')
  img = mpimg.imread(img_path)
  sp.set_title(data, fontsize=10)
  plt.imshow(img, cmap='gray')

plt.show()

# Generating Training , Validation & Testing Batches

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (150,150),
    subset = 'training',
    batch_size = 32,
    class_mode = 'binary'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    subset = 'validation',
    batch_size = 32,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'binary'
)


In [ ]:
from IPython.display import display

display(train_generator.class_indices)

display(test_generator.class_indices)

# Build CNN Model

In [ ]:
model = Sequential()

# add the convolutional layer
# filters, size of filters, padding, activation_function, input_shape
model.add(Conv2D(32, (5,5), padding='SAME', activation='relu', input_shape=(150, 150, 3)))

# pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# place a dropout layer
model.add(Dropout(0.5))

# add another conv. layer
model.add(Conv2D(64, (5,5), padding='SAME', activation='relu'))

# pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# place a dropout layer
model.add(Dropout(0.5))

# Flatten the image to 1 dimensional array
model.add(Flatten())

# add a dense layer: amount of nodes, activation
model.add(Dense(256, activation='relu'))

# place a dropout layer
# 0.5 drop out rate is recommended, half input nodes will be dropped at each update
model.add(Dropout(0.5))

# output layer :D
model.add(Dense(1, activation='sigmoid'))

model.summary()

# Compile & Train Model


In [ ]:
model.compile(Adam(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=10
)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Training', 'Validation'])
plt.title('Training and Validation Accuracy')
plt.xlabel('epoch')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')

In [ ]:
# test your model with some images from your local computer to predict whether a patient is affected by COVID19 or not.
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()
for fn in uploaded.keys():
  path='/content/'+fn
  print(path)
  img = image.load_img(path , target_size=(150,150))
  x = image.img_to_array(img)
  x=np.expand_dims(x,axis=0)
  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(fn)
  if classes==0:
    print('Covid19')
  else:
    print('Normal')

In [ ]:
model.save('covid19_classification')

In [ ]:
!zip -r covid19_classification.zip covid19_classification/